In [1]:
## Importing Necessary Modules 

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
## getting the appropriate website parsed into hmtl
### Defining a function which takes in a url and outputs soup
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return(soup)


In [89]:
#format url based off the race 
def format_url(race = 'tour-de-france', year = 2023, stage = None):
    
    url_basic = 'https://www.procyclingstats.com/race/{}/{}'
    
    if stage:
        return url_basic.format(race, year) + '/stage-{}'.format(stage)
    
    return url_basic.format(race, year)


In [90]:
def extract_result(url):
    
    #soupify
    soup = get_soup(url)
    
    headers = []
    data = []
    
    #find rows
    rows = soup.find_all('tr')
    
    #generate header
    header_row = rows[0]
    for header in header_row.find_all('th'):
        headers.append(header) 
        data.append([])
    
    #generate rows
    for data_row in rows[1:]:
        row = data_row.find_all('td')
        for i in range(len(row)):
            if len(row) != len(data):
                continue
            else:
                data[i].append(row[i].text.strip())
    
    #put into dataframe
    data_dict = {}
    for i in range(len(headers)):
        data_dict[headers[i]] = data[i]
    df = pd.DataFrame(data_dict)
    return(df)


In [ ]:
all_stages_data = {}
stage_number = 1
url_basic = 'https://www.procyclingstats.com/race/tour-de-france/2023/stage-'
for stage_number in range(1,22):
    url = url_basic + str(stage_number)
    all_stages_data[str(stage_number)] = extract_result(url)

    
print(all_stages_data['20'])

In [ ]:
def get_race_info(url):
    
    soup = get_soup(url)
    
    ul_elements = soup.find('ul', class_='infolist')
    ul_points = ul_elements.find_all('li')
    
    results = {}
    
    for point in ul_points:
        divs = point.find_all('div')
        
        key = divs[0].text.strip().replace(':','')
        val = divs[1].text
        
        results[key] = val

    return results

url = format_url('tour-de-france', 2021, 8)
print(get_race_info(url))